# Model Reveiew

In [150]:
from math import sqrt

import einops
from einops.layers.torch import Reduce
import torch
import torch.nn as nn
import numpy as np


## Basic PyTorch

### nn.Dropout 
Each forward pass will zero out some of the elements of the input tensor with probability p. And it will scale the remaining elements by $\frac{1}{1-p}$.

In [151]:
p = 0.5

module = nn.Dropout(p=p)
module.training
inp = torch.ones(3,5)
print(f'scale: {1/(1-p)}')
print(f'before dropout:\n{inp}')
print(f'after droput:\n{module(inp)}')
# module(inp)

scale: 2.0
before dropout:
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
after droput:
tensor([[0., 2., 0., 2., 0.],
        [0., 2., 2., 2., 2.],
        [0., 0., 2., 2., 0.]])


使用 random 實作 dropout

In [152]:
import random

# 將 5 成的資料設成 0
dropout_rate = 0.5
# Example output containing 10 values
example_output = [0.27, -1.03, 0.67, 0.99, 0.05, 
                  -0.37, -2.01, 1.13, -0.07, 0.73]

# Repeat as long as necessary 
while True:
    # Randomly choose index and set value to 0
    index = random.randint(0, len(example_output) - 1)
    example_output[index] = 0
    
    # Count values that are exactly 0
    dropped_out = 0
    for value in example_output:
        if value == 0:
            dropped_out += 1
    
    # If required number of outputs is zeroed - leave the loop        
    if dropped_out / len(example_output) >= dropout_rate:
        break

print(example_output)


[0, -1.03, 0, 0.99, 0, -0.37, 0, 1.13, 0, 0.73]


### nn.Linear
轉換輸入與輸出之間的維度關係，x 是輸入，y是輸出
$$
y = xA^T + b
$$

In [153]:
# 將輸入 dimension 10 轉換成 output dimension 20
module = nn.Linear(in_features=10, out_features=20)
print(f'module:{module}') 

n_samples = 40
# 最後面的 10 是輸入的維度，前面是指定的 batch size
inp_2d = torch.randn(n_samples, 10) 
inp_3d = torch.randn(n_samples, 33, 10)
inp_5d = torch.randn(n_samples, 2, 3, 4, 5, 10)
# inp_5d_false = torch.randn(n_samples, 2, 3, 4, 5, 20)

print(f'module(inp_2d).shape: {module(inp_2d).shape}')
print(f'module(inp_3d).shape: {module(inp_3d).shape}')
print(f'module(inp_5d).shape: {module(inp_5d).shape}')


module:Linear(in_features=10, out_features=20, bias=True)
module(inp_2d).shape: torch.Size([40, 20])
module(inp_3d).shape: torch.Size([40, 33, 20])
module(inp_5d).shape: torch.Size([40, 2, 3, 4, 5, 20])


In [154]:
# 測試 nn.Linear(embed_size, embed_size) 的作用
# y = xA^T + b
embed_size = 64
linear = nn.Linear(embed_size, embed_size)
# 產生隨機的 x.shap==3 的資料，其中第一維是 batch size，第二維是序列長度，第三維是嵌入維度
x = torch.randn(32, 10, embed_size)
print(f"x: {x[0][0][0]}")
print(f"linear(x)[0][0][0]: {linear(x)[0][0][0]}")

x: 0.3019552528858185
linear(x)[0][0][0]: -0.3937355875968933


### LayerNorm
LayerNorm 會對輸入進行標準化，使得輸入的均值為 0，方差為 1。這有助於模型訓練，因為它可以使不同特徵的數值範圍保持一致，從而更容易學習權重。

In [155]:
# 輸入特徵是一維資料
inp_features = torch.tensor([0.5, 0.3, 0.7, 0.2, 0.6])

# print the input and the mean and variance
mean_inp = torch.mean(inp_features)
variance_inp = torch.var(inp_features, unbiased=False)
print(f'input: {inp_features}')
print(f'before LayerNorm mean: {mean_inp}, variance: {variance_inp}')

# LayerNorm
layer_norm = nn.LayerNorm([5]) # Layer Normalization 填入特徵的維度
output = layer_norm(inp_features)

# print the output and the mean and variance
mean_opt = torch.mean(output)
variance_opt = torch.var(output, unbiased=False)
print(output)
print(f'after LayerNorm mean: {mean_opt}, variance: {variance_opt}')

input: tensor([0.5000, 0.3000, 0.7000, 0.2000, 0.6000])
before LayerNorm mean: 0.46000003814697266, variance: 0.03439999744296074
tensor([ 0.2156, -0.8625,  1.2938, -1.4016,  0.7547],
       grad_fn=<NativeLayerNormBackward0>)
after LayerNorm mean: 0.0, variance: 0.9997094869613647


## Positional Encoding + Input Embedding

### Code 簡介
`cls_token = nn.Parameter(torch.randn(1, output_channels))`

這行程式碼的意思是創建一個可學習的類別標記(class token),並將其定義為模型的參數。讓我們分解這行程式碼:

- `torch.randn(1, output_channels)` 創建了一個形狀為 (1, output_channels) 的隨機張量。
    - `1` 表示這個張量只有一個元素,即類別標記。
    - `output_channels` 表示類別標記的維度,與嵌入的維度相同。
    - `torch.randn` 函數從標準常態分佈中隨機取樣值來初始化張量。
- `nn.Parameter(...)` 將這個隨機初始化的張量封裝為一個可學習的參數。
    - 通過將張量傳遞給 `nn.Parameter`,我們告訴 PyTorch 這個張量是模型的一部分,需要在訓練過程中進行優化和更新。
    - 在這種情況下,類別標記 `cls_token` 將作為模型的一個參數,在反向傳播期間根據梯度進行更新。
`cls_token = ...` 將這個可學習的參數賦值給 `cls_token` 變數,以便在模型的前向傳播中使用。

舉個例子,假設 output_channels 的值為 20,那麼 cls_token 將是一個形狀為 (1, 20) 的張量,表示一個 20 維的類別標記。在訓練過程中,這個類別標記將與嵌入的序列一起傳遞給模型,並與序列的其他部分一起進行優化。


`torch.cat([einops.repeat(self.cls_token, "n e -> b n e", b=x.shape[0]), embedded], dim=1)`

這行程式碼的目的是將類別標記 (`cls_token`) 重複 `x.shape[0]` 次,並將其與嵌入的序列 (`embedded`) 在第二個維度 (dim=1) 上進行串聯。

1. `einops.repeat(self.cls_token, "n e -> b n e", b=x.shape[0])`:
    
    - `self.cls_token` 是形狀為 `(1, output_channels)` 的類別標記張量。
        
    - `einops.repeat` 是一個函數,用於重塑和重複張量。它使用一種特殊的表示法來指定輸入和輸出的維度。
        
    - `"n e -> b n e"` 是重塑的表示法,其中:
        
        - `n` 表示類別標記的數量,這裡是 1。
            
        - `e` 表示類別標記的維度,即 `output_channels`。
            
        - `b` 表示批次大小,即 `x.shape[0]`。
            
    - `b=x.shape[0]` 指定了重複的次數,即批次大小。
        
    - 這行程式碼的作用是將類別標記 `cls_token` 重複 `x.shape[0]` 次,使其與批次中的每個樣本相對應。
        
2. `torch.cat([einops.repeat(...), embedded], dim=1)`:
    
    - `torch.cat` 是一個函數,用於在指定維度上串聯張量。
        
    - 這裡,我們將重複後的類別標記張量 `einops.repeat(...)` 和嵌入的序列張量 `embedded` 在第二個維度 (dim=1) 上進行串聯。
        
    - 串聯後的結果將是一個新的張量,形狀為 `(batch_size, sequence_length + 1, output_channels)`,其中第二維的長度增加了 1,**因為我們在序列的開頭添加了類別標記**。

### Positional Encoding 的特色

因為Transformer中沒有Conv跟Recurrent, 沒有東西可以表示token在序列中的"相對位置或是絕對位置"，也就是說Attention機制沒有考慮"順序" ，所以需要Positional Encoding。因此他在encoder和decoder最底層的input embedding加上positional encodings來表示位置

In [156]:
class LinearEmbedding(nn.Sequential):

    def __init__(self, input_channels, output_channels) -> None:
        super().__init__(*[
            nn.Linear(input_channels, output_channels),
            nn.LayerNorm(output_channels),
            nn.GELU()
        ])
        # 創建一個可學習的類別標記(class token)，因為輸入的 channel 是一維的，並將其定義為模型的參數(nn.Parameter 特有的功能)
        self.cls_token = nn.Parameter(torch.randn(1, output_channels))

    def forward(self, x):
        # 直接使用上面創建的 Linear -> LayerNorm -> GELU()
        embedded = super().forward(x)
        return torch.cat([einops.repeat(self.cls_token, "n e -> b n e", b=x.shape[0]), embedded], dim=1)


### GeLU
Gaussian Error Linear Unit (GELU) 是一種激活函數，它結合了線性函數和高斯誤差函數。GELU 的定義如下:
$$
\text{GELU}\left(x\right) = x{P}\left(X\leq{x}\right) = x\Phi\left(x\right) = x \cdot \frac{1}{2}\left[1 + \text{erf}(x/\sqrt{2})\right],
$$

可以近似成

$$
0.5x\left(1+\tanh\left[\sqrt{2/\pi}\left(x + 0.044715x^{3}\right)\right]\right)
$$

#### 電路
使用LUT進行激活函數

### LayerNorm

$\gamma,\ \beta$ 為可學習的參數，$\epsilon$ 為避免梯度消失而添加

$$
y = \frac{x - \mathbb{E}[x]}{\sqrt{\text{Var}[x] + \epsilon}} * \gamma + \beta
$$

$$
\text{Var}(X) = \frac{1}{n}\sum_{i=1}^{n}(x_i - \mu)^2,\quad \mu = \frac{1}{n}\sum_{i=1}^{n}x_i
$$

$\epsilon =  1\times10^-5$

#### 電路執行

1. 計算輸入陣列的平均值
2. 計算標準差
3. 對每個值減去平均值，然後除標準差，不需要加上 1e-5

In [157]:
signal = np.array([1.00E+00,7.58E-01,1.12E-01,0.00E+00,8.06E-02,7.85E-02,6.61E-02,4.96E-02])

# avg =  np.mean(signal)
sum_avg = 0
for i in range(len(signal)):
    sum_avg += signal[i]
avg = sum_avg / len(signal)

# var = np.var(signal)
sum_var = 0
for i in range(len(signal)):
    sum_var += (signal[i] - avg) ** 2
var =  sum_var / len(signal)

layer_norm = (signal - avg) / np.sqrt(var + 1e-5)
print(layer_norm)

signal_torch = torch.tensor(signal)
layer_norm_torch = nn.LayerNorm(signal_torch.size())
print(layer_norm_torch(signal_torch.float()))


[ 2.03811871  1.36422237 -0.43469098 -0.74657689 -0.52213043 -0.52797829
 -0.56250851 -0.60845599]
tensor([ 2.0381,  1.3642, -0.4347, -0.7466, -0.5221, -0.5280, -0.5625, -0.6085],
       grad_fn=<NativeLayerNormBackward0>)


### cls_token 是甚麼?

輸入是 batch size = 1 且有 186 個點的訊號(`Size=[186]`)，透過線性層輸出 embed_size = 192 個 channel(`Size=[1, 186, 192]`)，再產生對應批次和通道的 `cls_token`(`Size=[1,1,192]`)，最後透過 `torch.cat` 串接起來(`Size=[1, 187, 192]`)。 

In [158]:
class embed(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_channels, output_channels),
            nn.LayerNorm(output_channels),
            nn.GELU()
        )

    def forward(self, x):
        return self.layers(x)

# cls_token 是一個二維矩陣但是存了一筆一維陣列
# 假設 embed_size = 8, input_channels = 1, batch_size = 1
signal = torch.tensor([1.00E+00,7.58E-01,1.12E-01,0.00E+00,8.06E-02,7.85E-02,6.61E-02,4.96E-02])
batch_signal_channel = signal.unsqueeze(0).unsqueeze(2) # 在第0位置增加維度，在第2位置增加維度

embed = embed(1, 8)
embed_signal = embed(batch_signal_channel)

cls_token = nn.Parameter(torch.randn(1, 8))
cls_token_repeat = einops.repeat(cls_token, "n e -> b n e", b=batch_signal_channel.shape[0])

output = torch.cat(([cls_token_repeat, embed_signal]), dim=1)

In [159]:
from prettytable import PrettyTable

# Create a new PrettyTable instance
table = PrettyTable()

# Add the column headers
table.field_names = ["Variable", "Size", "Note"]

# 假設 embed_size = 8, batch_size = 1, signal length = 8
table.add_row(["signal", signal.size(), "input signal consists of 8 samples"])
table.add_row(["batch_signal_channel", batch_signal_channel.size(), "Size=[batch, 187, 1], Corresponds to batch, signal length, channel."])
table.add_row(["embed_signal", embed_signal.size(), "Through the first layer, the output is sent to embed_size neurons."])
table.add_row(["cls_token", cls_token.size(), "The number of embed_size determines the length of the token needed."])
table.add_row(["cls_token_repeat", cls_token_repeat.size(), "Batch size = 1"])
table.add_row(["output", output.size(), "concatenate the vector with dimension = 1."])


# Print the table
print(table)


+----------------------+-----------------------+---------------------------------------------------------------------+
|       Variable       |          Size         |                                 Note                                |
+----------------------+-----------------------+---------------------------------------------------------------------+
|        signal        |    torch.Size([8])    |                  input signal consists of 8 samples                 |
| batch_signal_channel | torch.Size([1, 8, 1]) | Size=[batch, 187, 1], Corresponds to batch, signal length, channel. |
|     embed_signal     | torch.Size([1, 8, 8]) |  Through the first layer, the output is sent to embed_size neurons. |
|      cls_token       |   torch.Size([1, 8])  | The number of embed_size determines the length of the token needed. |
|   cls_token_repeat   | torch.Size([1, 1, 8]) |                            Batch size = 1                           |
|        output        | torch.Size([1, 9, 8]) |

### Linearembedding 的輸入與輸出

In [160]:
 
input_channels = 1
output_channels = 192
batch_size = 1
sequence_length = 5
x = torch.randn(batch_size, sequence_length, input_channels)

# 第一維 4 表示批次大小。
# 第二維 6 表示串聯後的序列長度,其中包括 5 個原始序列元素和 1 個添加的類別標記。
# 第三維 20 表示嵌入的維度。
embedding = LinearEmbedding(input_channels, output_channels)
output = embedding(x)
print(f"LinearEmbedding:{embedding}")
print(f"x.shape: {x.shape}")
print(f"output.shape: {output.shape}")
# 5->6 因為增加一個類別標記
# 1->192，因為 Linear embedding 的 output channel 設定 192

LinearEmbedding:LinearEmbedding(
  (0): Linear(in_features=1, out_features=192, bias=True)
  (1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  (2): GELU(approximate='none')
)
x.shape: torch.Size([1, 5, 1])
output.shape: torch.Size([1, 6, 192])


## Residual connection

In [161]:
class ResidualAdd(torch.nn.Module):
    def __init__(self, block):
        super().__init__()
        self.block = block

    def forward(self, x):
        return x + self.block(x)

`ResuidalAdd`

In [162]:
import torch

# 定義一個簡單的神經網絡模塊，將輸入乘以2
class DoubleBlock(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * 2

# 使用ResidualAdd類來創建一個帶殘差連接的模塊
residual_block = ResidualAdd(DoubleBlock())

# 創建一個輸入張量
input_tensor = torch.tensor([1, 2, 3], dtype=torch.int8)

# 通過殘差模塊傳遞輸入
output = residual_block(input_tensor)

# 輸入 [1,2,3] 經過 DoubleBlock 得到 [2,4,6]，再加上輸入得到 [3,6,9]
# 此例子充分展示 Residual add 的功能，通過 x + self.block(x) 進行殘差連接
# x 是原本的輸入 tensor，self.block(x) 是經過模塊處理後的輸出
print(output)

tensor([3, 6, 9], dtype=torch.int8)


## Multi-Head-Attention

### 簡單說明 Multi-Head-Attention 用到的變數
這段代碼定義了一個名為 MultiHeadAttention 的 PyTorch 模組類別,用於實現多頭注意力機制。讓我們逐步解釋這段代碼:

1. `__init__` 方法初始化了模組的參數,包括三個線性投影層 (queries_projection, values_projection, keys_projection) 用於將輸入張量投影到 queries、keys 和 values 的空間中。另一個線性投影層 final_projection 用於最終的輸出。此外,還設置了 embed_size 和 num_heads 參數。
2. `forward` 方法定義了模組的前向傳播行為。它首先檢查輸入張量 x 的維度是否為 3 (批次大小、序列長度、嵌入維度)。
接下來,輸入張量 x 被投影到 keys、values 和 queries 的空間中。
3. 使用 `einops` 庫,keys、values 和 queries 被重新排列為多頭表示,其中每個頭對應一個特定的子空間。
4. 計算 `queries` 和 keys 之間的點積,得到 energy_term。
5. `energy_term` 被縮放以防止極端的 softmax 值,然後經過 softmax 運算得到 mh_out。
6. 使用 `torch.einsum` 計算加權和,將 mh_out 與 values 相乘並求和,得到每個頭的輸出。
7. 所有頭的輸出被連接起來,然後通過 `final_projection` 層得到最終的輸出張量。

總的來說,這個模組實現了標準的多頭注意力機制,將輸入序列映射到一組注意力加權的表示,捕獲了不同子空間中的重要信息。這種注意力機制被廣泛應用於諸如 Transformer 等自然語言處理模型中。

### Attention 的運作原理

#### Attention 簡介
- 注意力function做的事情可被描述為mapping一個query和一群key-value的pairs到輸出 (Q, K, V, Output都是vector)
- Output is computed as a weighted sum of the values，values的weight則是由query和其相對應的key所算出
- Q,K,V 在文字翻譯上而言， Q是在找哪個字的key vector可能會貢獻我的語意最多 , K是這個字可以貢獻給哪個字最多語意, V是最後的輸出，也就是這個字的語意是什麼。
- 換句話說就是
    - Q : to match other
    - K : to be matched
    - V : information to be extracted
    - Attention: 吃兩個向量，輸出一個分數來代表這兩個向量有多匹配、多相關
- Self Attention: 拿每個q去對每個k做Attention (Scaled Dot-product attention)


#### Transformer 的 Scaled Dot-Product Attention
一樣是做 Q 和 K 的內積，內積以矩陣乘法表示的話就是 $QK^{T}$，但是 Q, K 都是 $d_k$ 維度的輸入，也就是 embedded size。內積完的結果除 $\sqrt{d_k}$，再送入 softmax 就是 attention 的結果。**需要注意的是 Q的計算和這個 attention 的計算是同時進行的(硬體 Pipeline 優化的方向)** 

> 研究員懷疑 $d_k$ 很大的時候dot product的規模太大，導致softmax後的數值太小 (extremely small gradients), 才會加開根號緩解

$$
Attention(Q, K, V) = \mathrm{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

### Multi-Head Attention : Transformer 最重要的機制
![](https://imgur-backup.hackmd.io/4xMlLna.png)
- 很多個 self-attention concat 後所得出的結果
- Multi-head attention 用來計算相對全部的字而言當前這個字能給予多少的資訊量 (讓每個head都能學到不同feature的特徵)
- Transformer 原文使用 8 個 heads，每個 head 的維度是 64，也就是 embedded size。但是每個 head 都有降維，實際上的計算不會和 single-head 差太多。

> Self-attention layer in Encoder: 在這一層中 所有的key, values, queries都是從同一個地方來的, 他們都是從前一層的encoder的output來的

### FFN (Feed Forward Network) MLP
也就是在 `model.py` 中的 MLP，FFN 通常由兩個線性變換(Linear)和一個 ReLU(GeLU) 組成，層和層之間(不同的head之間)是用不同的參數丟入FFN之中。

> FFN 可看成kernel size=1的2個 Conv


### Residual Connection

![](https://imgur-backup.hackmd.io/JBqsUsH.png)

`TransformerEncoderLayer` 描述的就是 encoder 的灰色方框，由Multi-head self-attention 和 FC Feed Forward所組成
，其中每兩個 sublayers 之間用Residual connection連接。

In [163]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, embed_size, num_heads, attention_store=None):
        super().__init__()
        # 通過 nn.Linear 相當於做通過全連接層
        self.queries_projection = nn.Linear(embed_size, embed_size)
        self.values_projection = nn.Linear(embed_size, embed_size)
        self.keys_projection = nn.Linear(embed_size, embed_size)
        self.final_projection = nn.Linear(embed_size, embed_size)
        self.embed_size = embed_size
        self.num_heads = num_heads

    def forward(self, x):
        """
        :param x: Size=[batch, seq_len, embed_size]
        :return: 
        """
        assert len(x.shape) == 3
        keys = self.keys_projection(x)
        values = self.values_projection(x)
        queries = self.queries_projection(x)
        keys = einops.rearrange(keys, "b n (h e) -> b n h e", h=self.num_heads)
        queries = einops.rearrange(queries, "b n (h e) -> b n h e", h=self.num_heads)
        values = einops.rearrange(values, "b n (h e) -> b n h e", h=self.num_heads)
        # q, k 做 dot-product -> self-attention
        energy_term = torch.einsum("bqhe, bkhe -> bqhk", queries, keys)
        divider = sqrt(self.embed_size)
        mh_out = torch.softmax(energy_term, -1)
        out = torch.einsum('bihv, bvhd -> bihd ', mh_out / divider, values)
        out = einops.rearrange(out, "b n h e -> b n (h e)")
        return self.final_projection(out)
    

class MLP(nn.Sequential):
    def __init__(self, input_channels, expansion=4):
        super().__init__(*[
            nn.Linear(input_channels, input_channels * expansion),
            nn.GELU(),
            nn.Linear(input_channels * expansion, input_channels)
        ])
        
        
class TransformerEncoderLayer(torch.nn.Sequential):
    def __init__(self, embed_size=768, expansion=4, num_heads=8, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__(
            *[
                ResidualAdd(nn.Sequential(*[
                    nn.LayerNorm(embed_size),
                    MultiHeadAttention(embed_size, num_heads),
                    nn.Dropout(dropout)
                ])),
                ResidualAdd(nn.Sequential(*[
                    nn.LayerNorm(embed_size),
                    MLP(embed_size, expansion),
                    nn.Dropout(dropout)
                ]))
            ]
        )

`einops.rearrange` 

- b 表示批次大小。
- n 表示序列長度。
- h 表示頭的數量，這裡是 self.num_heads。
- e 表示每個頭的嵌入維度，這裡是 embed_size / num_heads。

將輸入張量 keys 的形狀從 (batch_size, sequence_length, embed_size) 變為 (batch_size, sequence_length, num_heads, embed_size / num_heads)。這樣做的目的是將嵌入維度 embed_size 分割成 num_heads 個子空間，每個子空間的維度是 embed_size / num_heads。舉例來說，假設我們有一個形狀為 (32, 10, 64) 的張量，num_heads 為 8。那麼這行程式碼將會將這個張量的形狀變為 (32, 10, 8, 8)。這意味著我們將 **64 維的嵌入空間分割成了 8 個 8 維的子空間，每個子空間對應一個頭**。

## Multi-Head-Attention Calculation

### 查看 b n (h e) -> b n h e 如何拆解


- `b` represents the batch size.
-  `q` and `k` represent the sequence length, but they are different dimensions because one is for queries and the other is for keys.
-  `h` represents the number of attention heads.
-  `e` represents the embedding size.

In [164]:

class Model(nn.Module):
    
    def __init__(self, embed_size):
        super(Model, self).__init__()
        self.linear = nn.Linear(embed_size, embed_size)
    def forward(self, x):
        out = self.linear(x)
        return out

In [165]:
import torch

# 假設我們有一個批次大小為 32，序列長度為 10，頭數為 8，嵌入維度為 64 的模型
batch_size = 1
seq_length = 186
num_heads = 6
embed_size = 192

# 創建隨機張量 queries 和 keys
Linear = Model(embed_size)
queries = Linear(torch.randn(batch_size, seq_length, embed_size))
print(f"queries.shape: {queries.shape}")
print(queries[0][0][32:64]) # 查看 queries [0, 186, 192] 中 186 的第 0 個的 32 到 64 的值
queries_head_embed = einops.rearrange(queries, "b n (h e) -> b n h e", h=num_heads)
print(queries_head_embed[0][0].shape)# 查看 queries [0, 186, 192] 中 186 的第 0 個
print(queries_head_embed[0][0][1]) # 查看 qu

# (h e) -> (h e) 是按照矩陣的順序拆解的

queries.shape: torch.Size([1, 186, 192])
tensor([-0.4552, -1.1705, -0.0840,  0.7994, -0.2486,  0.4171, -0.7824, -0.3609,
        -0.5861, -0.4159,  0.5052, -0.4152,  0.0929,  1.2724, -0.8910,  0.1214,
        -0.4691,  1.1529,  0.1098, -0.2445,  0.7564, -0.3510,  0.2852, -1.2411,
         0.3565, -0.3883,  0.0108,  0.7387, -0.4135, -0.0317,  0.2832,  0.4710],
       grad_fn=<SliceBackward0>)
torch.Size([6, 32])
tensor([-0.4552, -1.1705, -0.0840,  0.7994, -0.2486,  0.4171, -0.7824, -0.3609,
        -0.5861, -0.4159,  0.5052, -0.4152,  0.0929,  1.2724, -0.8910,  0.1214,
        -0.4691,  1.1529,  0.1098, -0.2445,  0.7564, -0.3510,  0.2852, -1.2411,
         0.3565, -0.3883,  0.0108,  0.7387, -0.4135, -0.0317,  0.2832,  0.4710],
       grad_fn=<SelectBackward0>)


### 計算 queries 和 keys 的點積

`energy_term`: Size = (batch_size, seq_length_q, num_heads, seq_length_k)。假設 batch_size = 1，第 h 個 head 中第 q 個 query 和第 k 個 key 的點積等於 `energy_term[0][q][h][k]`。

In [39]:
import numpy as np
import einops

# Assume queries_head_embed and keys_head_embed have shape (1, 186, 6, 32)
batch_size, seq_len, num_heads, embed_size = 1, 186, 6, 32

# Create random numpy arrays for queries_head_embed and keys_head_embed
queries_head_embed = np.random.randn(batch_size, seq_len, num_heads, embed_size)
keys_head_embed = np.random.randn(batch_size, seq_len, num_heads, embed_size)

# bqhd,bkhd->bqhk 針對每個 head 做 q,k 的 dot product
energy_term_einsum = np.einsum('bqhd,bkhd->bqhk', queries_head_embed, keys_head_embed)
energy_term_head_zero_q0_k1 = sum(queries_head_embed[0][0][0][:] * keys_head_embed[0][1][0][:])
energy_term_head_zero_q185_k185 = sum(queries_head_embed[0][185][0][:] * keys_head_embed[0][185][0][:])

print(f"energy_term.shape: {energy_term_einsum.shape}") 
print("=============================================================")


# 第零個 head 的第一個 query 和第一個 key 的點積
print("第零個 head 的第一個 query 和第一個 key 的點積")
energy_term_head_zero_q0_k0 = sum(queries_head_embed[0][0][0][:] * keys_head_embed[0][0][0][:])
print(f"energy_term_head_zero_q0_k0: {energy_term_head_zero_q0_k0}")  # Output: (1, 186, 6, 186)
print(f"energy_term[0][0][0][0]: {energy_term_einsum[0][0][0][0]}")
print("=============================================================")
      
# 第零個 head 的第一個 query 和第二個 key 的點積
print("第零個 head 的第一個 query 和第二個 key 的點積")
print(f"energy_term_head_zero_q0_k1: {energy_term_head_zero_q0_k1}")  # Output: (1, 186, 6, 186)
print(f"energy_term[0][0][0][1]: {energy_term_einsum[0][0][0][1]}")  
print("=============================================================")

# 第零個 head 的第 186 個 query 和第 186 個 key 的點積
print("第零個 head 的第 186 個 query 和第 186 個 key 的點積")
print(f"energy_term_head_zero_q185_k185: {energy_term_head_zero_q185_k185}")  # Output: (1, 186, 6, 186)
print(f"energy_term[0][0][0][185]: {energy_term_einsum[0][185][0][185]}") 
print("=============================================================")

# 第二個 head 的第 130 個 query 和第 100 個 key 的點積
print("第二個 head 的第 130 個 query 和第 100 個 key 的點積")
energy_term_head_three_q130_k100 = sum(queries_head_embed[0][130][2][:] * keys_head_embed[0][100][2][:])
print(f"energy_term_head_three_q130_k100: {energy_term_head_three_q130_k100}")
print(f"energy_term[0][130][2][100]: {energy_term_einsum[0][130][2][100]}")


energy_term.shape: (1, 186, 6, 186)
第零個 head 的第一個 query 和第一個 key 的點積
energy_term_head_zero_q0_k0: -7.70901092448986
energy_term[0][0][0][0]: -7.709010924489862
第零個 head 的第一個 query 和第二個 key 的點積
energy_term_head_zero_q0_k1: -2.6244431567384368
energy_term[0][0][0][1]: -2.624443156738433
第零個 head 的第 186 個 query 和第 186 個 key 的點積
energy_term_head_zero_q185_k185: -17.152041727777256
energy_term[0][0][0][185]: -17.15204172777725
第二個 head 的第 130 個 query 和第 100 個 key 的點積
energy_term_head_three_q130_k100: 5.714561656594404
energy_term[0][130][2][100]: 5.7145616565944035


### Multi-Head Attention Output

In [121]:
import torch
import einops

batch_size, seq_len, num_heads, embed_size = 1, 186, 6, 32
# energy_term.shape=[1,186,6,186]       
energy_term = torch.randn(1, 186, 6, 186)
mh_out = torch.softmax(energy_term, -1)
mh_out_scale = mh_out / 13.85464 # -1 代表隊最後一個維度做 softmax

# embedding size = 192, sqrt(embedding size) = 13.85464
values = torch.randn(batch_size, seq_len, num_heads, embed_size)
out_torch = torch.einsum('bihv, bvhd -> bihd ', mh_out_scale , values)

# 每個頭的計算過程
print(mh_out_scale[0, :, 0, :].shape) # 這個操作會直接從4維張量中選取第一個batch的所有第二維度元素,以及所有第四維度元素,結果是一個2維張量。
print(values[0, :, 0, :].shape) # 這個操作會直接從4維張量中選取第一個batch的所有第二維度元素,以及所有第四維度元素,結果是一個2維張量。
out_matmul_head_zero = torch.matmul(mh_out_scale[0, :, 0, :], values[0, :, 0, :])
out_matmul_head_one = torch.matmul(mh_out_scale[0, :, 1, :], values[0, :, 1, :])
out_matmul_head_two = torch.matmul(mh_out_scale[0, :, 2, :], values[0, :, 2, :])
out_matmul_head_three = torch.matmul(mh_out_scale[0, :, 3, :], values[0, :, 3, :])
out_matmul_head_four = torch.matmul(mh_out_scale[0, :, 4, :], values[0, :, 4, :])
out_matmul_head_five = torch.matmul(mh_out_scale[0, :, 5, :], values[0, :, 5, :])

out_matmul_heads = torch.stack([ out_matmul_head_zero, out_matmul_head_one, out_matmul_head_two,
    out_matmul_head_three, out_matmul_head_four, out_matmul_head_five], dim=1)

out_matmul_heads = out_matmul_heads.unsqueeze(0)

print(f"out_torch.shape: {out_torch.shape}")
print(f"out_matmul.shape: {out_matmul_heads.shape}")

print(f"out_torch == out_matmul_heads? {torch.allclose(out_torch, out_matmul_heads)}")
# out = einops.rearrange(out, "b n h e -> b n (h e)")

torch.Size([186, 186])
torch.Size([186, 32])
out_torch.shape: torch.Size([1, 186, 6, 32])
out_matmul.shape: torch.Size([1, 186, 6, 32])
out_torch == out_matmul_heads? True


In [137]:
# reshape 
# print(out_matmul_heads.shape())
# print(out_matmul_heads[0,:,0,:])
# print(f"out_matmul_heads.shape: {out_matmul_heads[0,:,0,:].shape}")

print(f"out_matmul_heads.shape: {out_matmul_heads.shape}")
# 實際算法
out_matmul_heads_manual_re = torch.cat((out_matmul_heads[0,:,0,:], out_matmul_heads[0,:,1,:], out_matmul_heads[0,:,2,:],
                             out_matmul_heads[0,:,3,:], out_matmul_heads[0,:,4,:], out_matmul_heads[0,:,5,:]), dim=1)
out_matmul_heads_re= einops.rearrange(out_matmul_heads,"b n h e -> b n (h e)")

out_matmul_heads_manual_re = out_matmul_heads_manual_re.unsqueeze(0)
print(f"out_matmul_heads_re.shape: {out_matmul_heads_re.shape}")
print(f"out_matmul_heads_manual_re == out_matmul_heads_re? {torch.allclose(out_matmul_heads_manual_re, out_matmul_heads_re)}")



out_matmul_heads.shape: torch.Size([1, 186, 6, 32])
out_matmul_heads_re.shape: torch.Size([1, 186, 192])
out_matmul_heads_manual_re == out_matmul_heads_re? True
